In [ ]:
# https://stacspec.org/en/tutorials/1-read-stac-python/
# https://stacspec.org/en/about/tools-resources/

# https://stacindex.org/catalogs/usgs-landsat-collection-2-api#/

In [22]:
import json

from pystac_client import Client
from pystac import Catalog, get_stac_version
from pystac.extensions.eo import EOExtension
from pystac.extensions.label import LabelExtension

In [2]:
# Read the example catalog
root_catalog = Catalog.from_file('https://raw.githubusercontent.com/stac-utils/pystac/main/docs/example-catalog/catalog.json')

In [3]:
root_catalog.describe()


* <Catalog id=landsat-stac-collection-catalog>
    * <Collection id=landsat-8-l1>
      * <Item id=LC80140332018166LGN00>
      * <Item id=LC80150322018141LGN00>
      * <Item id=LC80150332018189LGN00>
      * <Item id=LC80300332018166LGN00>


In [4]:
# Print some basic metadata from the Catalog
print(f"ID: {root_catalog.id}")
print(f"Title: {root_catalog.title or 'N/A'}")
print(f"Description: {root_catalog.description or 'N/A'}")

ID: landsat-stac-collection-catalog
Title: STAC for Landsat data
Description: STAC for Landsat data


In [5]:
print(get_stac_version())

1.0.0


In [6]:
collections = list(root_catalog.get_collections())

print(f"Number of collections: {len(collections)}")
print("Collections IDs:")
for collection in collections:
    print(f"- {collection.id}")

Number of collections: 1
Collections IDs:
- landsat-8-l1


In [7]:
collection = root_catalog.get_child("landsat-8-l1")
if collection is None:
    print("Collection is Empty. Check your downloads and try again.")
else:
    print("Collection has a root child. You may proceed to the following steps.")

Collection has a root child. You may proceed to the following steps.


In [8]:
items = list(root_catalog.get_all_items())

print(f"Number of items: {len(items)}")
for item in items:
    print(f"- {item.id}")

Number of items: 4
- LC80140332018166LGN00
- LC80150322018141LGN00
- LC80150332018189LGN00
- LC80300332018166LGN00


In [9]:
item = root_catalog.get_item("LC80140332018166LGN00", recursive=True)

In [10]:
item.geometry

{'type': 'Polygon',
 'coordinates': [[[-76.12180471942207, 39.95810181489563],
   [-73.94910518227414, 39.55117185146004],
   [-74.49564725552679, 37.826064511480496],
   [-76.66550404911956, 38.240699151776084],
   [-76.12180471942207, 39.95810181489563]]]}

In [11]:
item.bbox

[-76.66703, 37.82561, -73.94861, 39.95958]

In [12]:
item.datetime

datetime.datetime(2018, 6, 15, 15, 39, 9, tzinfo=tzutc())

In [13]:
item.collection_id

'landsat-8-l1'

In [14]:
item.get_collection()

<Collection id=landsat-8-l1>

In [15]:
print(item.common_metadata.instruments)
print(item.common_metadata.platform)
print(item.common_metadata.gsd)

['OLI_TIRS']
landsat-8
30


In [16]:
item.stac_extensions

['https://stac-extensions.github.io/eo/v1.1.0/schema.json',
 'https://stac-extensions.github.io/view/v1.0.0/schema.json',
 'https://stac-extensions.github.io/projection/v1.1.0/schema.json']

In [17]:
EOExtension.has_extension(item)

True

In [18]:
LabelExtension.has_extension(item)

False

In [19]:
eo_item_ext = EOExtension.ext(item)
eo_item_ext.cloud_cover

22

In [20]:
item.properties['eo:cloud_cover']

22

In [21]:
for asset_key in item.assets:
    asset = item.assets[asset_key]
    print('{}: {} ({})'.format(asset_key, asset.href, asset.media_type))

index: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/index.html (text/html)
thumbnail: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_thumb_large.jpg (image/jpeg)
B1: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B1.TIF (image/tiff)
B2: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B2.TIF (image/tiff)
B3: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B3.TIF (image/tiff)
B4: https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B4.TIF (image/tiff)
B5: https://s3-us

In [23]:
asset = item.assets['B3']
asset.to_dict()

{'href': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/014/033/LC08_L1TP_014033_20180615_20180703_01_T1/LC08_L1TP_014033_20180615_20180703_01_T1_B3.TIF',
 'type': 'image/tiff',
 'title': 'Band 3 (green)',
 'eo:bands': [{'name': 'B3',
   'full_width_half_max': 0.06,
   'center_wavelength': 0.56,
   'common_name': 'green'}],
 'roles': []}

In [24]:
eo_asset_ext = EOExtension.ext(asset)
bands = eo_asset_ext.bands
bands

[<Band name=B3>]

In [25]:
bands[0].to_dict()

{'name': 'B3',
 'full_width_half_max': 0.06,
 'center_wavelength': 0.56,
 'common_name': 'green'}

# 2

In [23]:
catalog_url = "https://catalogue.dataspace.copernicus.eu/stac"

In [24]:
catalog = Client.open(url=catalog_url)

In [25]:
catalog

<Client id=Catalogue>

In [ ]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [14.335072274700622, 40.88868507222068],
            [14.335072274700622, 40.7572458523517],
            [14.524378202519358, 40.7572458523517],
            [14.524378202519358, 40.88868507222068],
            [14.335072274700622, 40.88868507222068],
        ]
    ],
}
time_of_interest = "2023-07-01/2023-07-31"

search = catalog.search(
    collections=["SENTINEL-2"],
    intersects=area_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 10}},
)

In [ ]:
search

In [ ]:
items = search.item_collection()
print(f"Returned {len(items)} Items")

# 3

In [ ]:
# https://documentation.dataspace.copernicus.eu/APIs/STAC.html#search-for-items-by-attributes

In [26]:
client = Client.open(url="https://catalogue.dataspace.copernicus.eu/stac")

In [27]:
for collection in client.get_collections():
    print(collection)

<CollectionClient id=COP-DEM>
<CollectionClient id=S2GLC>
<CollectionClient id=TERRAAQUA>
<CollectionClient id=SENTINEL-3>
<CollectionClient id=SENTINEL-5P>
<CollectionClient id=SENTINEL-1-RTC>
<CollectionClient id=SENTINEL-1>
<CollectionClient id=SMOS>
<CollectionClient id=LANDSAT-7>
<CollectionClient id=CCM>
<CollectionClient id=LANDSAT-5>
<CollectionClient id=ENVISAT>
<CollectionClient id=LANDSAT-8-ESA>
<CollectionClient id=SENTINEL-6>
<CollectionClient id=GLOBAL-MOSAICS>
<CollectionClient id=SENTINEL-2>
